In [17]:
import zipfile
import pandas as pd

# Replace 'your_zip_file.zip' with the path to your actual ZIP file.
zip_path = "D:\en-sw.txt (2).zip"


<>:5: SyntaxWarning: invalid escape sequence '\e'
<>:5: SyntaxWarning: invalid escape sequence '\e'
C:\Users\hp\AppData\Local\Temp\ipykernel_10644\2114745633.py:5: SyntaxWarning: invalid escape sequence '\e'
  zip_path = "D:\en-sw.txt (2).zip"


In [23]:

with zipfile.ZipFile(zip_path, 'r') as z:
    # Read English sentences
    with z.open('ELRC-wikipedia_health.en-sw.en') as f_eng:
        english_text = f_eng.read().decode('utf-8')
        english_sentences = english_text.strip().splitlines()
# 
    # Read Swahili translations
    with z.open('ELRC-wikipedia_health.en-sw.sw') as f_swa:
        swahili_text = f_swa.read().decode('utf-8')
        swahili_sentences = swahili_text.strip().splitlines()


In [25]:

# Check if both files have the same number of lines
if len(english_sentences) != len(swahili_sentences):
    raise ValueError("The number of English sentences and Swahili translations do not match!")

# Create a DataFrame with the two columns.
df = pd.DataFrame({
    'English sentence': english_sentences,
    'Swahili Translation': swahili_sentences
})



In [27]:
df

,English sentence,Swahili Translation
0,"In 1983, two separate research groups led by R...","Mnamo 1983, vikundi viwili tofauti vya watafit..."
1,"In the early days, the CDC did not have an off...","Katika siku za kwanza, kituo hicho hakikuwa na..."
2,The chance of quitting is improved with social...,Uwezekano wa kukoma huendelezwa kwa usaidizi w...
3,Both strains of the tuberculosis bacteria shar...,Wote Matatizo ya bakteria kifua kikuu kushirik...
4,Products with 60% to 95% alcohol by volume are...,Michanganyiko huwa na asilimia 60-95 za alikoh...
...,...,...
602,It is responsible for about 1–5% of cases and ...,Hatari hii huchangia takriban asilimia 1-5 ya ...
603,In the most socioeconomically deprived parts o...,"Katika sehemu nyingi maskini za nchi hiyo, mtu..."
604,Since the WHO's staging system does not requir...,Kwa vile mfumo wa uainishaji wa SAD hauhitaji ...
605,The relative contributions of these two factor...,Vinavyochangia visababishi hivi viwili hutofau...


In [33]:

# Save the DataFrame to a CSV file.
df.to_csv('test_en_to_swa.csv', index=False)
print("CSV file 'test_en_to_swa.csv' has been created successfully.")

CSV file 'test_en_to_swa.csv' has been created successfully.


In [35]:
df_new=pd.read_csv(r"C:\Users\hp\NLP_PROJECT\test_en_to_swa.csv")

In [37]:

# Remove missing values
df_new = df_new.dropna()

# Remove duplicate sentence pairs
df_new = df_new.drop_duplicates()

# Strip extra spaces from both columns
df_new["English sentence"] = df_new["English sentence"].str.strip()
df_new["Swahili Translation"] = df_new["Swahili Translation"].str.strip()

# Remove sentences that are too short
df_new = df_new[df_new["English sentence"].str.len() > 1]
df_new = df_new[df_new["Swahili Translation"].str.len() > 1]

# Convert to lowercase
df_new["English sentence"] = df_new["English sentence"].str.lower()
df_new["Swahili Translation"] = df_new["Swahili Translation"].str.lower()

# Function to clean text by removing numbers and unwanted characters
def clean_text(text):
    text = re.sub(r"[^a-zA-Z.,?!'’ ]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df_new["English sentence"] = df_new["English sentence"].apply(clean_text)
df_new["Swahili Translation"] = df_new["Swahili Translation"].apply(clean_text)

# Function to format sentences properly
def format_sentence(text):
    if len(text) == 0:
        return text
    text = text.capitalize()
    if text[-1] not in ".?!":
        text += "."
    return text

df_new["English sentence"] = df_new["English sentence"].apply(format_sentence)
df_new["Swahili Translation"] = df_new["Swahili Translation"].apply(format_sentence)

# Optionally, compute word counts and filter out extreme length ratios as done previously
df_new["english_length"] = df_new["English sentence"].apply(lambda x: len(x.split()))
df_new["swahili_length"] = df_new["Swahili Translation"].apply(lambda x: len(x.split()))
df_new["length_ratio"] = df_new["english_length"] / df_new["swahili_length"]
df_new = df_new[(df_new["length_ratio"] < 3) & (df_new["length_ratio"] > 0.3)]

# Drop helper columns
df_new = df_new.drop(columns=["english_length", "swahili_length", "length_ratio"])

# Final check for non-empty sentences
df_new = df_new[(df_new["English sentence"].str.len() > 1) & (df_new["Swahili Translation"].str.len() > 1)]

In [39]:
df_new

,English sentence,Swahili Translation
0,"In , two separate research groups led by rober...","Mnamo , vikundi viwili tofauti vya watafiti vi..."
1,"In the early days, the cdc did not have an off...","Katika siku za kwanza, kituo hicho hakikuwa na..."
2,The chance of quitting is improved with social...,Uwezekano wa kukoma huendelezwa kwa usaidizi w...
3,Both strains of the tuberculosis bacteria shar...,Wote matatizo ya bakteria kifua kikuu kushirik...
4,Products with to alcohol by volume are effecti...,Michanganyiko huwa na asilimia za alikoholi.
...,...,...
602,It is responsible for about of cases and the c...,Hatari hii huchangia takriban asilimia ya visa...
603,In the most socioeconomically deprived parts o...,"Katika sehemu nyingi maskini za nchi hiyo, mtu..."
604,Since the who's staging system does not requir...,Kwa vile mfumo wa uainishaji wa sad hauhitaji ...
605,The relative contributions of these two factor...,Vinavyochangia visababishi hivi viwili hutofau...


In [41]:

# Save the DataFrame to a CSV file.
df.to_csv('test_en_to_swa_cleaned.csv', index=False)